In [8]:
from scripts.tools.issuetracker import *
import os
import shutil
from officelib import wordlib
import dateutil.parser
import dateutil.tz
import datetime

In [261]:

def iter_gantt_outline(gantt):
    seen = set()
    level = 0
    for i in gantt:
        seen.clear()
        p = i
        level = 0
        while True:
            if p in seen:
                raise ValueError("Circular graph detected: %s" % seen)
            seen.add(p)
            p = p.parent
            if p is None:
                break
            level += 1
        yield level, i
                
            
def make_tree(gantt):
    tree = {}

    for iss in gantt:
        path = [iss]
        p = iss.parent
        while p is not None:
            path.append(p)
            p = p.parent
        v = tree
        while path:
            i = path.pop()
            if i not in v:
                v[i] = {}
            v = v[i]
    return tree


def append_line(lvl, issue, domain):
    nonlocal lines
    link = "https://" + domain + "/issues/%d" % issue.id
    lines.append((lvl, "Issue #%d: %s" % (issue.id, issue.subject), link))


def parse_tree(tree, lines, domain, level=0):
    for k in tree:
        append_lines(level, k, domain)
        parse_tree(tree[k], append, level+1)
            

def create_outline(gantt, domain):
    
    lines = []
    tree = make_tree(gantt)
    parse_tree(tree, lines, domain, 0)
    return lines
    
    
# def create_outline(gantt, domain, set_gantt):
#     lines = []
    
#     def append(lvl, issue):
#         link = "https://" + domain + "/issues/%d" % issue.id
#         lines.append((lvl, "Issue #%d: %s" % (issue.id, issue.subject), link))
        
#     for lvl, issue in iter_gantt_outline(gantt):
#         p = issue.parent
#         plvl = lvl - 1
#         while p is not None and p not in set_gantt:
#             set_gantt.add(p)
#             append(plvl, p)
#             p = p.parent
#             plvl -= 1
#         append(lvl, issue)
#     return lines

In [262]:
def make_style(doc, name="PY_FRS_TITLE", font_size=24, font="Calibri", 
                     alignment=None, indent=None, tabstops=None):
    if alignment is None:
        alignment = wordlib.c.wdAlignParagraphCenter
    style = doc.Styles.Add(name)
    style.Font.Size = font_size
    style.Font.Name = font
    style.ParagraphFormat.Alignment = alignment
    style.ParagraphFormat.SpaceBeforeAuto = False
    style.ParagraphFormat.SpaceAfterAuto = False
    style.ParagraphFormat.LineSpacingRule = wordlib.c.wdLineSpaceSingle
    if indent is not None:
        style.ParagraphFormat.LeftIndent = wordlib.inches_to_points(indent)
    style.NoSpaceBetweenParagraphsOfSameStyle = True
    if tabstops is not None:
        style.ParagraphFormat.TabStops.Add(wordlib.inches_to_points(tabstops), wordlib.c.wdAlignTabLeft, wordlib.c.wdTabLeaderDots)
    return style

In [263]:
def move(r, n):
    r.MoveStart(wordlib.c.wdCharacter, n)

def add_index_lines(doc, r, lines):
    styles = {}
    nlines = len(lines)
    for i, (lvl, itxt, href) in enumerate(lines, 1):
        print("\rCreating index %d of %d       " % (i, nlines), end="")
        style_name = "PY_FRS_BODY_%d" % lvl
        if style_name not in styles:
            indent = lvl * 0.1
            style = make_style(doc, style_name, 12, "Calibri", wordlib.c.wdAlignParagraphLeft, indent, 6)
            styles[style_name] = style
        else:
            style = styles[style_name]
        
        r.Text = itxt + "\t"
        r.Style = style

        move(r, len(itxt) + 2)
        doc.Hyperlinks.Add(r, href, "", href, "Link")
        r.InsertAfter("\r")
        move(r, 20)  # enough for "link"
        doc.Paragraphs.Add()
    print()

def create_word_index(lines, doc, index_title, from_date, to_date):
    
    print("Creating Word Document...")
    
    pgs = doc.Paragraphs
    
    # title
    r = pgs(1).Range
    r.Text = index_title
    r.Style = make_style(doc, "PY_FRS_TITLE", 24, "Calibri")
    r.InsertAfter("\r")
    
    #subtitle
    move(r, len(index_title) + 2)
    ds1 = "%d/%d/%d" % (from_date.month, from_date.day, from_date.year)
    ds2 = "%d/%d/%d" % (to_date.month, to_date.day, to_date.year)
    txt = "Updated Specifications from %s to %s" % (ds1, ds2)
    r.Text = txt
    r.Style = make_style(doc, "PY_FRS_SUBTITLE", 12)
    r.InsertAfter("\r")
    
    # index
    move(r, len(txt) + 2)
    add_index_lines(doc, r, lines)
    

def create_pdf_index(gantt, domain, index_title, from_date, to_date, set_gantt):
    print("Creating PDF Index...")
    lines = create_outline(gantt, domain)
    word = wordlib.Word()
    doc = word.Documents.Add()
    with wordlib.lock_screen(word):
        create_word_index(lines, doc, index_title, from_date, to_date)
    #create_word_index(lines, doc, index_title, from_date, to_date)
    

In [264]:
def should_include_issue(iss, from_date, to_date):
    return (iss.sprint_milestone.name == '3.0' and 
            iss.tracker.name == 'Specification' and
            from_date <= iss.updated_on <= to_date)
    
def parse_date(date):
    if isinstance(date, str):
        dt = dateutil.parser.parse(date)
        if dt.tzinfo is None:
            return datetime.datetime(dt.year, dt.month, dt.day, tzinfo=dateutil.tz.tzutc())
        return dt
    return date

def today():
    td = datetime.datetime.today()
    return datetime.datetime(td.year, td.month, td.day, tzinfo=dateutil.tz.tzutc())

In [265]:
def main(domain, user, password, from_date=None, to_date=None, index_title="FRS Update"):
    
    if from_date is None:
        from_date = datetime.datetime(1900, 1, 1, tzinfo=dateutil.tz.tzutc())
    else:
        from_date = parse_date(from_date)

    if to_date is None or to_date.lower() == "today":
        to_date = today()
    else:
        from_date = parse_date(to_date)
        
    
    api = IssuetrackerAPI(domain, user, password)
    gantt = api.download_issues('pbssoftware').values()
    gantt = [i for i in gantt if should_include_issue(i, from_date, to_date)]
    set_gantt = set(gantt)
    create_pdf_index(gantt, domain, index_title, from_date, to_date, set_gantt)
    #return gantt
    

In [266]:
main("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist', "2016-6-1", "today", 'Hello v3.0 FRS Update')

Creating PDF Index...
Creating Word Document...
Creating index 34 of 34       


In [151]:
#from scripts.tools.issuetracker import IssuetrackerAPI
#api a= IssuetrackerAPI("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist')
#issues = api.download_issues('pbssoftware')
#gantt = issues.values()
#gantt2 = [i for i in gantt if should_include_issue(i, from_date, to_date)]

In [152]:
class IssuetrackerAPI():
    def __init__(self, *args):
        pass
    def download_gantt(self, *args):
        return gantt
    def download_issues(self, *args):
        return issues

In [153]:
from_date = parse_date("2016-6-1")
to_date = today()

In [154]:
len(gantt)

671

In [141]:
g=iter_gantt_outline(gantt)
filtered = [i for i in gantt if should_include_issue(i, from_date, to_date)]
g2=iter_gantt_outline(filtered)
for i in gantt:
    if should_include_issue(i, from_date, to_date):
        pass
        #print(i.id)
x, y = list(zip(*g2))
iter_gantt_outline(filtered)

<generator object iter_gantt_outline at 0x000001C8A39049E8>

In [177]:
oldrepr = Issue.__repr__

In [217]:
newrepr = lambda self: "Issue(%d)" % self.id
Issue.__repr__ = Issue.__str__ = newrepr

In [218]:
_oldrepr = oldrepr

In [247]:
def make_tree(gantt):
    tree = {}

    for iss in gantt:
        path = [iss]
        p = iss.parent
        while p is not None:
            path.append(p)
            p = p.parent
        v = tree
        while path:
            i = path.pop()
            if i not in v:
                v[i] = {}
            v = v[i]
    return tree


def parse_tree(tree, append, level=0):
    for k in tree:
        append(level, k)
        parse_tree(tree[k], append, level+1)
            

def create_outline(gantt, domain):
    print("Create_outline")
    lines = []
    
    def append(lvl, issue):
        nonlocal lines
        link = "https://" + domain + "/issues/%d" % issue.id
        lines.append((lvl, "Issue #%d: %s" % (issue.id, issue.subject), link))
    
    tree = make_tree(gantt)
    print(tree)
    lines = parse_tree(tree, append, 0)
    return lines

In [248]:
create_outline(gantt, 'issue.pbsbiotech.com')

Create_outline
{Issue(1921): {}, Issue(2591): {}, Issue(2042): {}, Issue(2579): {}, Issue(2978): {}, Issue(3242): {}, Issue(1230): {}, Issue(1959): {}, Issue(2958): {}, Issue(2534): {}, Issue(2956): {}, Issue(2931): {}, Issue(2059): {}, Issue(2607): {}, Issue(3058): {}, Issue(3178): {}, Issue(3018): {}, Issue(3253): {}, Issue(3237): {}, Issue(3245): {}, Issue(2536): {}, Issue(2092): {}, Issue(2802): {}, Issue(2598): {}, Issue(3250): {}, Issue(2557): {}, Issue(3040): {}, Issue(2798): {}, Issue(2847): {}, Issue(2655): {}, Issue(2692): {}, Issue(1843): {}, Issue(2952): {}, Issue(2513): {}, Issue(3167): {}, Issue(1678): {}, Issue(2329): {}, Issue(3075): {}, Issue(2744): {}, Issue(2087): {}, Issue(2573): {}, Issue(3107): {}, Issue(2248): {}, Issue(1842): {}, Issue(2722): {}, Issue(2044): {}, Issue(1758): {}, Issue(2717): {}, Issue(3108): {}, Issue(2843): {}, Issue(3109): {}, Issue(2938): {}, Issue(2640): {}, Issue(1098): {}, Issue(2980): {}, Issue(2610): {}, Issue(3163): {}, Issue(3067): {}

In [235]:
tree=make_tree(filtered)
p=parse_tree(tree)
for lvl, id in p:
    print(" "*lvl, id)

 2712
  2851
  1845
  989
  2825
  2828
  2725
   1073
   2908
  2814
  3001
   2859
   2819
   2922
   2856
  2758
  2808
  1885
 2716
  1583
  2787
 2857
  2880
 2876


In [232]:
tree=make_tree(filtered)
p=parse_tree(tree)
for lvl, id in p:
    print(" "*lvl, id)

 2712
  2851
  1845
  989
  2825
  2828
  2725
   1073
   2908
  2814
  3001
   2859
   2819
   2922
   2856
  2758
  2808
  1885
 2716
  1583
  2787
 2857
  2880
 2876


In [243]:
issues[2715].parent

Issue(2712)